W sklepie ze zdrową żywnością wytwarza się produkty dżem jabłkowy i mus jabłkowy. Wytworzenie jednostki produktu I wymaga zużycia 8 jabłek i 5 łyżek miodu, zaś jednostki musu - 2 jabłek i 5 łyżek miodu. Dostawy surowców w każdym dniu wynosza dla jabłek i miodu odpowiednio 40 i 25 jednostek. Dżem sprzedaje się po 12 zł a mus po 11 zł. Jabłko kosztuje 0.3 zł, łyżka miodu 0.2. 

Określ dzienną produkcję każdego produktu tak, aby otrzymać maksymalny zysk. Dla dwóch przypadków:

- ( HURT) wielkość produkcji można wyrazić jako nieujemne liczby rzeczywiste ( np 1.2 kg dżemu)

- (DETAL) wielkości produkcji mierzy się w jednostkach niepodzielnych ( 1 słoik dżemu). W tym przypadku do każdej wyprodukowanej jednostki trzeba doliczyć opakowanie za 0.5 zł


## Rozwiązanie

Słownie:

Zysk = Zarobek na dzemie + zarobek na musie - koszt dzemu - koszt musu - dodatkowy koszt

Zmienne decyzyjne:

Ilość ( masa lub ilosc sztuk) wyprodukowanych dzemow i musuw

Ograniczenia:

Ograniczone dostawy

Znamy ile kosztuje mus i drzem



In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

import pulp

print(pulp.__version__)

In [4]:
from pulp import *
import pandas as pd

Definiowanie problemu

In [ ]:
prob = LpProblem("Najlepsza produkcja HURT",LpMaximize)

# 2 zmienne
dzem=LpVariable("Dzem",0) 
mus=LpVariable("Mus",0)
# pomocnicze zmienne
zuzycie_jablek =LpVariable("Jablka",0)
zuzycie_miodu =LpVariable("Miod",0)

prob += 16* dzem + 11* mus - 0.3 * zuzycie_jablek - 0.2 * zuzycie_miodu
prob += 8 * dzem+ 2 * mus == zuzycie_jablek, "Zuzycie jablek"
prob += 5 * dzem + 5 * mus == zuzycie_miodu, "Zuzycie miodu"
prob += zuzycie_jablek <= 40, "Maksimum jablek"
prob += zuzycie_miodu <= 25, "Maksimum miodu"


Rozwiązanie problemu

In [ ]:
prob.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Zysk wyniesie {} zł  ".format( value(prob.objective)))


Dodawanie sliderów do problemu - szybka modyfikacja online wartości więzów w problemie

In [5]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

style = {'description_width': 'initial'}

cena_jablek_slider = FloatSlider(min=0,max = 1, value= 0.3, description="cena_jablek_slider", style= style )
cena_miodu_slider = FloatSlider(min=0,max = 1,value= 0.2, description="cena_miodu_slider", style= style )
zysk_z_dzemu_slider = FloatSlider(min=0,max = 50, value= 16, description="zysk_z_dzemu_slider", style= style )
zysk_z_musu_slider = FloatSlider(min=0,max = 50, value= 11, description="zysk_z_musu_slider", style= style )
zuzycie_jablek_dzem_slider = FloatSlider(min=0,max = 20,value= 8, description="zuzycie_jablek_dzem_slider", style= style )
zuzycie_jablek_mus_slider = FloatSlider(min=0,max = 20,value= 2, description="zuzycie_jablek_mus_slider", style= style )
zuzycie_miodu_dzem_slider = FloatSlider(min=0,max = 20,value= 5, description="zuzycie_miodu_dzem_slider", style= style )
zuzycie_miodu_mus_slider = FloatSlider(min=0,max = 20,value= 5, description="zuzycie_miodu_mus_slider", style= style )
maksimum_jablek_slider = IntSlider(min=0,max = 200,value= 40, description="maksimum_jablek_slider", style= style )
maksimum_miodu_slider = FloatSlider(min=0,max = 200,value= 25, description="maksimum_miodu_slider", style= style )


def hurt_produkcja(cena_jablek = 0.3,
    cena_miodu = 0.2,
    zysk_z_dzemu = 16,
    zysk_z_musu = 11,
    zuzycie_jablek_dzem = 8,
    zuzycie_jablek_mus = 2,
    zuzycie_miodu_dzem = 5,
    zuzycie_miodu_mus = 5,
    maksimum_jablek = 40,
    maksimum_miodu = 25):
    
    prob = LpProblem("Najlepsza produkcja HURT",LpMaximize)

    # 2 zmienne
    dzem=LpVariable("Dzem",0) 
    mus=LpVariable("Mus",0)
    # pomocnicze zmienne
    zuzycie_jablek =LpVariable("Jablka",0)
    zuzycie_miodu =LpVariable("Miod",0)

    prob += zysk_z_dzemu * dzem + zysk_z_musu * mus - cena_jablek * zuzycie_jablek - cena_miodu * zuzycie_miodu
    prob += zuzycie_jablek_dzem * dzem + zuzycie_jablek_mus * mus == zuzycie_jablek, "Zuzycie jablek"
    prob += zuzycie_miodu_dzem * dzem + zuzycie_miodu_mus * mus == zuzycie_miodu, "Zuzycie miodu"
    prob += zuzycie_jablek <= maksimum_jablek, "Maksimum jablek"
    prob += zuzycie_miodu <= maksimum_miodu, "Maksimum miodu"

    
    prob.solve()
    # The status of the solution is printed to the screen
    print("Status:", LpStatus[prob.status])
    # Each of the variables is printed with it's resolved optimum value
    for v in prob.variables():
        print(v.name, "=", v.varValue)
    print("Zysk wyniesie {} zł  ".format( value(prob.objective)))
    
    shadows = [{'name':name, 'shadow price': c.pi, "slack": c.slack} for name, c in prob.constraints.items()]
    print("*"*10+"Shadow Prices"+"*"*10)
    print(pd.DataFrame(shadows))
    
    
interact(hurt_produkcja,cena_jablek = cena_jablek_slider,
    cena_miodu =    cena_miodu_slider,
    zysk_z_dzemu =    zysk_z_dzemu_slider,
    zysk_z_musu =zysk_z_musu_slider,
    zuzycie_jablek_dzem =zuzycie_jablek_dzem_slider,
    zuzycie_jablek_mus =zuzycie_jablek_mus_slider,
    zuzycie_miodu_dzem =zuzycie_miodu_dzem_slider,
    zuzycie_miodu_mus =zuzycie_miodu_mus_slider,
    maksimum_jablek =maksimum_jablek_slider,
    maksimum_miodu =maksimum_miodu_slider)


interactive(children=(FloatSlider(value=0.3, description='cena_jablek_slider', max=1.0, style=SliderStyle(desc…

<function __main__.hurt_produkcja>

Przykłady sliderów

In [6]:
from ipywidgets import IntSlider

style = {'description_width': 'initial'}
IntSlider(description='A too long description', style=style)

IntSlider(value=0, description='A too long description', style=SliderStyle(description_width='initial'))

In [7]:

x_widget = FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=x_widget, y=y_widget)

interactive(children=(FloatSlider(value=0.0, description='x', max=10.0, step=0.05), FloatSlider(value=5.0, des…

<function __main__.printer>